# RFM-анализ клиентской базы аптечной сети

Короткое саммари

Здесь должно быть концептуальное описание ваших действий:

- как выбирали метрики для анализа
- почему именно такие метрики
- какие ваши основные гипотезы
- ваш ход рассуждений
- Содержательная часть

Здесь будет непосредственно ваша аналитика:

- словесное описание запроса и цель, которую хотите достичь
- код запроса
- описание полученных результатов и промежуточные выводы
- любая другая важная на ваш взгляд информация
- И так для каждого запроса.

Выводы

Здесь должны быть финальные выводы:


- что у вас получилось
- подтвердились ли ваши гипотезы
- какие бизнес-решения вы предлагаете принять и почему

Важно: Из вашего отчета все члены команды 
(руководство, другие аналитики, менеджеры) должны понять - как им развивать компанию дальше.

Основной мотив исследования такой: скоро сезон спадет и выручки в аптеке начнут падать. У ребят есть механизм СМС-рассылок, но они не хотят тратить бюджеты впустую - они хотят провести персонализированную рассылку.

Идея такая: нет смысла предлагать персональную скидку для повышения среднего чека тем, кто вообще покупал один раз и то аскорбинку.

Соответственно, цели исследования:


- Провести классификацию пользователей

- Сформировать рекомендации - какой группе пользователей что предлагать

- Для решения этой задачи вам предлагается базово сосредоточится на RFM-анализе.

Основная проблема при проведении такого исследования - подобрать оптимальные пороги. Например, каких клиентов относить в группу 1 по признаку Recency, а кого в группу 2. И здесь мы оставляем вам пространство для  ̶к̶р̶е̶а̶т̶и̶в̶а̶ исследования. Вы сами будете выбирать методы, с помощью которых определите этот порог. Мы можем лишь посоветовать:

- Не забывайте про ABC
- Средние, медианы и перцентили хорошо показывают распределение той или иной величины
- С помощью графика boxplot можно также оценить распределение величин
- Самое главное - принимайте решение обоснованно, опираясь на данные, а не потому что «так захотелось».



* Recency — давность
* Frequency — частота
* Monetary - деньги

Для начала перед проведением самого RFM анализа стоит выбрать метрики:
для отбора товаров значительно важных по прибыли и количеству мы возьмем ABC анализ  
для разбиения под категорий в RFM анализе, возьмем медиану

1. F=3 приносит больше выручки, чем F=1 → (высокая частота = выручка).
2. M=3 покупают больше A-товаров.
3. M=1 — C-товары.
4. F=3 → больше позиций в чеке.
5. Сегмент 3-3-3 генерирует непропорционально высокую выручку.
6. R=3 чаще покупать B-товары (дополняющие покупки).
7. Клиенты с высоким Monetary получают больше скидок.
8. Клиенты из разных аптек отличаются по F и M.
9. A-товары дают основную выручку → подтверждение ABC.
10. C-товары часто покупаются разовыми клиентами → подтверждение поведения.